In [13]:
pc_number=1

In [2]:
import os
os.chdir('../')

In [3]:
import pandas as pd
from pathlib import Path
from PIL import Image
import json
import random
import subprocess

In [14]:
root_path = Path('./data/')

In [15]:
style_df = pd.read_csv(root_path / 'metadata' / 'style.csv')
style_df = style_df[~style_df['File_name'].isna()]
style_df = style_df.sort_values('File_name').reset_index(drop=True)

content_df = pd.read_csv(root_path / 'metadata' / 'content.csv')
content_df = content_df[~content_df['File_name'].isna()]
content_df = content_df.sort_values('File_name').reset_index(drop=True)

content_df.shape, style_df.shape

((116, 8), (59, 8))

In [16]:
elements = content_df.shape[0]//5
print(elements)
content_df = content_df.loc[pc_number*elements:pc_number*elements + elements -1]

23


In [17]:
weights = [1e4, 5e4, 10e4]

start = 1 + pc_number*elements*style_df.shape[0]*len(weights)

In [18]:
def cartesian_product(d):
    index = pd.MultiIndex.from_product(d.values(), names=d.keys())
    return pd.DataFrame(index=index).reset_index()

prod = cartesian_product({
    'content': content_df['File_name'],
    'style': style_df['File_name'],
    'weight': weights,
})

prod['index'] = list(prod.index+start)
prod['done'] = False

prod.to_csv(f'status_{pc_number}.csv', index=False)

In [19]:
content_path = root_path / 'content-images'
style_path = root_path / 'style-images'

metadata_path = root_path / 'output' / 'metadata'
metadata_path.mkdir(exist_ok=True, parents=True)

In [20]:
print(f'Starting from name {start}')

for i, row in prod.iterrows():
    content_img_path = content_path / row['content']
    style_img_path = style_path / row['style']
    weight = row['weight']
    index = str(row['index'])
    
    content = content_df.loc[content_df['File_name'] == row['content']]
    style = style_df.loc[style_df['File_name'] == row['style']]
    
    print(f"Processing content image: {row['content']}")
    print(f"Processing style: {row['style']}")
    print(f"Processing weight: {row['weight']}")
    print(f"Processing output name: {index}")
    
    #!python neural_style_transfer.py --content_img_name {row['content']} --style_img_name {row['style']} --output_img_name {index} --style_weight {str(weight)}
        
    metadata = {
        "description": "The most iconic pieces of art, reimagined by AI.",
        "image": "TBD",
        "name" : f"{content['Title']} X {style['Title']}",
        "animation_url": "TBD",
        "attributes": [
            {
                "trait_type": "Content",
                "value": content['Title']
            },
            {
                "trait_type": "Content Author",
                "value": content['Author']
            },
            {
                "trait_type": "Style",
                "value": style['Title']
            },
            {
                "trait_type": "Style Author",
                "value": style['Author']
            },
            {
                "trait_type": "Orientation",
                "value": content['Orientation']
            },
            {
                "trait_type": "File Name",
                "value": output_name
            },
            {
                "trait_type": "Style weight",
                "value": weight
            }
        ]
    }
    
    # file_metadata_path = metadata_path / (output_name + '.json')
    # with open(file_metadata_path, 'w') as f:
    #     json.dump(metadata, f)
    
    prod.loc[i, 'done'] = True
    if i%100==0:
        prod.to_csv(f'status_{pc_number}.csv', index=False)

    print('\n\n')

Starting from name 4072
Processing content image: dimanche_a_midi.jpg
Processing style: alpine_pasture.jfif
Processing weight: 10000.0
Processing output name: 4072



Processing content image: dimanche_a_midi.jpg
Processing style: alpine_pasture.jfif
Processing weight: 50000.0
Processing output name: 4073



Processing content image: dimanche_a_midi.jpg
Processing style: alpine_pasture.jfif
Processing weight: 100000.0
Processing output name: 4074



Processing content image: dimanche_a_midi.jpg
Processing style: animals_in_a_landscape.jpg
Processing weight: 10000.0
Processing output name: 4075



Processing content image: dimanche_a_midi.jpg
Processing style: animals_in_a_landscape.jpg
Processing weight: 50000.0
Processing output name: 4076



Processing content image: dimanche_a_midi.jpg
Processing style: animals_in_a_landscape.jpg
Processing weight: 100000.0
Processing output name: 4077



Processing content image: dimanche_a_midi.jpg
Processing style: bed_of_chrysanthemums.jpg
Proce